In [1]:
import pandas as pd
from keybert import KeyBERT
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from konlpy.tag import Okt

# Okt 형태소 분석기 초기화
okt = Okt()

# 불용어 리스트
stop_words = ['좋다', '입다', '같다', '이다', '있다', '입']

# 전처리 함수 정의
def clean_review_column(df, column_name):
    df[column_name] = df[column_name].str.replace('\n', ' ', regex=False)  # 개행 문자 제거
    df[column_name] = df[column_name].str.replace(r'[ㄱ-ㅎㅏ-ㅣ]+', ' ', regex=True)  # 한글 자모 제거
    df[column_name] = df[column_name].str.replace(r'[^\w\s]', ' ', regex=True)  # 구두점 제거
    df[column_name] = df[column_name].str.replace(r'\s+', ' ', regex=True)  # 여러 공백을 한 칸으로 변환
    return df

# 키워드 원형 변환 함수
def lemmatize_keyword(keyword):
    morphs = okt.pos(keyword, stem=True)  # 형태소 분석 및 어간 추출
    lemmatized = [word for word, pos in morphs if pos in ['Noun', 'Verb', 'Adjective']]  # 명사, 동사, 형용사만 추출
    return ' '.join(lemmatized) if lemmatized else None  # 원형 변환된 단어들을 합침, 없으면 None 반환

# 핵심 키워드 추출 함수 정의 (KeyBERT)
def extract_keywords(text):
    kw_model = KeyBERT()  # KeyBERT 모델 초기화
    vectorizer = CountVectorizer(ngram_range=(1, 1))  # 단일 단어 추출
    keywords = kw_model.extract_keywords(text, vectorizer=vectorizer, top_n=5)  # 상위 5개의 키워드 추출
    return keywords

# 전체 리뷰에서 키워드 추출 및 처리
def process_keywords(df, review_column):
    # 전처리
    df = clean_review_column(df, review_column)
    documents = df[review_column]

    # 각 리뷰에 대해 핵심 키워드 추출
    df['Keywords'] = documents.apply(extract_keywords)

    # 키워드를 모두 모아서 리스트로 풀기
    all_keywords = []
    for keywords in df['Keywords']:
        for keyword, score in keywords:
            lemmatized_keyword = lemmatize_keyword(keyword)
            if lemmatized_keyword and lemmatized_keyword not in stop_words:  # 불용어 필터링
                all_keywords.append(lemmatized_keyword)

    # 각 키워드의 빈도를 계산하고 상위 10개의 키워드 추출
    keyword_counts = Counter(all_keywords)
    top_10_keywords = keyword_counts.most_common(10)

    # 키워드만 리스트로 변환
    top_10_keywords_list = [keyword for keyword, _ in top_10_keywords]
    keywords_str = ', '.join(top_10_keywords_list)

    return keywords_str

# 데이터프레임에 새로운 행을 추가하는 함수
def add_to_dataframe(df, csv_title, keywords_str):
    new_row = pd.DataFrame({'title': [csv_title], 'keyword': [keywords_str]})
    df = pd.concat([df, new_row], ignore_index=True)
    return df

# 전체 워크플로우 함수
def process_csv_file(csv_file, csv_title, df):
    # CSV 파일 읽기
    new_df = pd.read_csv(csv_file)

    # 상위 10개 키워드 추출
    keywords_str = process_keywords(new_df, 'Review')

    # 데이터프레임에 새로운 데이터 추가
    df = add_to_dataframe(df, csv_title, keywords_str)
    return df

# 초기 데이터프레임 생성 (첫 CSV 처리)
csv_title = "[SET] 워셔블 케이블 반팔 니트 세트"
csv_file = "[SET] 워셔블 케이블 반팔 니트 세트.csv"
df = pd.DataFrame(columns=['title', 'keyword'])  # 빈 데이터프레임 생성
df = process_csv_file(csv_file, csv_title, df)

# 이후 새로운 CSV 파일을 처리할 때
# 새로운 CSV 파일과 제목을 추가할 수 있습니다.
# 예시:
# new_csv_title = "새로운 제품 제목"
# new_csv_file = "새로운 파일 경로.csv"
# df = process_csv_file(new_csv_file, new_csv_title, df)

# 데이터프레임 저장
# df.to_csv('output.csv', index=False)

C:\Users\HWAN\anaconda3\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
df

,title,keyword
0,[SET] 워셔블 케이블 반팔 니트 세트,"사이즈, 색감, 여름, 가성 비, 재질, 가격, 배송, 니트, 만족하다, 시원하다"


In [2]:
# 전처리 코드 정의
def clean_review_column(df, column_name):
    # '\n'을 제거하고 한글 자모 및 구두점 제거
    df[column_name] = df[column_name].str.replace('\n', ' ', regex=False)  # 개행 문자 제거
    df[column_name] = df[column_name].str.replace(r'[ㄱ-ㅎㅏ-ㅣ]+', ' ', regex=True)  # 한글 자모 제거
    df[column_name] = df[column_name].str.replace(r'[^\w\s]', ' ', regex=True)  # 특정 구두점 제거
    
    # 여러 공백을 한 칸으로 변환
    df[column_name] = df[column_name].str.replace(r'\s+', ' ', regex=True)  # 여러 개의 공백을 한 칸으로 변환

    return df

In [3]:
import pandas as pd

df = pd.read_csv('[SET] 워셔블 케이블 반팔 니트 세트.csv')
df = clean_review_column(df, 'Review')
documents = df['Review']

In [4]:
documents

0      여름에 꼭 가지고 있어야 템 중 하나 입니다 원플원임에도 불구하고 퀄리티랑 핏이 너...
1                   여름에 입기 딱 좋은 두께에요 부드러워서 맨 몸에 입어도 괜찮아요
2      시원한 느낌이라 좋습니다 부드러워 편하네요 카라와 단추 라인이 예쁘게 잘 나왔습니다...
3      무신사 매장까지 방문해서 재질확인후이 구매하였습니다 여름에 입기 좋은 두께감입니다 ...
4         최고에요 사이즈 살짝 오버핏인데 그냥 너무 좋야요 제 스펙이랑 비슷하시면 m 사세요
                             ...                        
885                          합리적인 가격이고 여름 반팔은 역시나 수아레네요 
886                         옷 2벌 와요 선택하실때 다른거로 선택하시길 바래요
887           약간 큰 느낌이긴 한데 이 가격에 이정도면 무난 합니다 한여름에는 힘들듯해요
888                      사이즈도 좋고 여름에 잘 입고 다니고 있습니다 추천합니다
889                       가성비 젛아요 품은 생각보다 좀 커요 참고하세요 이뻐요
Name: Review, Length: 890, dtype: object

In [5]:
from keybert import KeyBERT
from sklearn.feature_extraction.text import CountVectorizer

# KeyBERT 모델 초기화
kw_model = KeyBERT()

vectorizer = CountVectorizer(ngram_range=(1, 1))

# 각 문서에서 핵심 키워드 추출
def extract_keywords(text):
    # 각 텍스트에서 상위 5개의 키워드 추출
    keywords = kw_model.extract_keywords(text, vectorizer=vectorizer, top_n=5)
    return keywords

# 전체 문서에 대해 핵심 키워드 추출
df['Keywords'] = documents.apply(extract_keywords)

# 키워드가 잘 추출되었는지 확인
print(df[['Review', 'Keywords']].head())

C:\Users\HWAN\anaconda3\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange



                                              Review  \
0  여름에 꼭 가지고 있어야 템 중 하나 입니다 원플원임에도 불구하고 퀄리티랑 핏이 너...   
1               여름에 입기 딱 좋은 두께에요 부드러워서 맨 몸에 입어도 괜찮아요   
2  시원한 느낌이라 좋습니다 부드러워 편하네요 카라와 단추 라인이 예쁘게 잘 나왔습니다...   
3  무신사 매장까지 방문해서 재질확인후이 구매하였습니다 여름에 입기 좋은 두께감입니다 ...   
4     최고에요 사이즈 살짝 오버핏인데 그냥 너무 좋야요 제 스펙이랑 비슷하시면 m 사세요   

                                            Keywords  
0  [(정사이즈로, 0.3535), (구매하시면, 0.3386), (구매하시거나, 0....  
1  [(입어도, 0.6102), (부드러워서, 0.4709), (여름에, 0.4678)...  
2  [(적당히, 0.6256), (라인이, 0.6139), (시원한, 0.6104), ...  
3  [(재질확인후이, 0.7081), (방문해서, 0.6456), (무신사, 0.594...  
4  [(스펙이랑, 0.6513), (사이즈, 0.5276), (사세요, 0.4755),...  


In [6]:
df

,Review,Star,Keywords
0,여름에 꼭 가지고 있어야 템 중 하나 입니다 원플원임에도 불구하고 퀄리티랑 핏이 너...,5.0,"[(정사이즈로, 0.3535), (구매하시면, 0.3386), (구매하시거나, 0...."
1,여름에 입기 딱 좋은 두께에요 부드러워서 맨 몸에 입어도 괜찮아요,5.0,"[(입어도, 0.6102), (부드러워서, 0.4709), (여름에, 0.4678)..."
2,시원한 느낌이라 좋습니다 부드러워 편하네요 카라와 단추 라인이 예쁘게 잘 나왔습니다...,5.0,"[(적당히, 0.6256), (라인이, 0.6139), (시원한, 0.6104), ..."
3,무신사 매장까지 방문해서 재질확인후이 구매하였습니다 여름에 입기 좋은 두께감입니다 ...,5.0,"[(재질확인후이, 0.7081), (방문해서, 0.6456), (무신사, 0.594..."
4,최고에요 사이즈 살짝 오버핏인데 그냥 너무 좋야요 제 스펙이랑 비슷하시면 m 사세요,5.0,"[(스펙이랑, 0.6513), (사이즈, 0.5276), (사세요, 0.4755),..."
...,...,...,...
885,합리적인 가격이고 여름 반팔은 역시나 수아레네요,5.0,"[(합리적인, 0.6039), (가격이고, 0.5589), (반팔은, 0.4801)..."
886,옷 2벌 와요 선택하실때 다른거로 선택하시길 바래요,5.0,"[(선택하시길, 0.6377), (2벌, 0.521), (다른거로, 0.5048),..."
887,약간 큰 느낌이긴 한데 이 가격에 이정도면 무난 합니다 한여름에는 힘들듯해요,5.0,"[(한여름에는, 0.6743), (무난, 0.6062), (합니다, 0.5799),..."
888,사이즈도 좋고 여름에 잘 입고 다니고 있습니다 추천합니다,5.0,"[(추천합니다, 0.8154), (다니고, 0.6586), (사이즈도, 0.6375..."


In [7]:
from collections import Counter

# 각 문서에서 추출된 키워드를 모은 리스트 생성
all_keywords = []

for keywords in df['Keywords']:
    # keywords는 [(단어, 가중치), (단어, 가중치), ...] 형식이므로 단어만 추출
    all_keywords.extend([keyword[0] for keyword in keywords])

# 키워드의 빈도 계산
keyword_counts = Counter(all_keywords)

# 빈도 상위 10개의 키워드 출력
top_keywords = keyword_counts.most_common(15)

# 상위 10개의 키워드와 그 빈도를 출력
print("상위 15개의 키워드:")
for word, count in top_keywords:
    print(f"{word}: {count}번")

상위 15개의 키워드:
너무: 91번
입기: 60번
가성비: 49번
좋아요: 46번
여름에: 44번
사이즈도: 44번
만족합니다: 41번
색감도: 39번
입고: 35번
생각보다: 34번
사이즈: 30번
마음에: 28번
입을: 27번
재질도: 27번
좋고: 27번


In [8]:
from collections import Counter
from konlpy.tag import Okt

# Okt 형태소 분석기 초기화
okt = Okt()

stop_words = ['좋다', '입다', '같다', '이다', '있다', '입']

# 키워드를 원형으로 변환
def lemmatize_keyword(keyword):
    morphs = okt.pos(keyword, stem=True)  # 형태소 분석 및 어간 추출
    lemmatized = [word for word, pos in morphs if pos in ['Noun', 'Verb', 'Adjective']] # 명사, 동사, 형용사 추출
    return ' '.join(lemmatized) if lemmatized else None  # 원형 변환된 단어들을 합침, 없으면 None 반환

# 키워드를 모두 모아서 리스트로 풀기
all_keywords = []
for keywords in df['Keywords']:
    # 키워드 원형으로 변환 후 리스트에 추가 (불용어 제외)
    for keyword, score in keywords:
        lemmatized_keyword = lemmatize_keyword(keyword)
        if lemmatized_keyword and lemmatized_keyword not in stop_words:  # 불용어 필터링
            all_keywords.append(lemmatized_keyword)

# 각 키워드의 빈도를 계산
keyword_counts = Counter(all_keywords)

# 상위 10개의 키워드 추출
top_10_keywords = keyword_counts.most_common(10)

# 결과 출력
print(top_10_keywords)

[('사이즈', 114), ('색감', 92), ('여름', 72), ('가성 비', 62), ('재질', 59), ('가격', 58), ('배송', 56), ('니트', 55), ('만족하다', 53), ('시원하다', 52)]


In [23]:
csv_title = "[SET] 워셔블 케이블 반팔 니트 세트"

# 키워드만 추출하여 리스트로 변환
top_10_keywords_list = [keyword for keyword, _ in top_10_keywords]

keywords_str = ', '.join(top_10_keywords_list)

# 데이터프레임 생성 (2열 1행: 'title'과 'keyword')
df = pd.DataFrame({
    'title': [csv_title],
    'keyword': [keywords_str]
})

In [24]:
df

,title,keyword
0,[SET] 워셔블 케이블 반팔 니트 세트,"사이즈, 색감, 여름, 가성 비, 재질, 가격, 배송, 니트, 만족하다, 시원하다"


### 유사도

In [9]:
from collections import defaultdict

# 키워드와 유사도를 저장할 딕셔너리 초기화
keyword_scores = defaultdict(float)

# 키워드를 모두 모아서 유사도를 합산
for keywords in df['Keywords']:
    for keyword, score in keywords:
        keyword_scores[keyword] += score  # 키워드에 유사도(score) 더하기

# 유사도가 높은 상위 10개 키워드 추출
top_10_keywords_with_scores = sorted(keyword_scores.items(), key=lambda x: x[1], reverse=True)[:10]

# 결과 출력
print(top_10_keywords_with_scores)


[('너무', 50.934399999999975), ('입기', 29.485300000000002), ('만족합니다', 29.198499999999996), ('가성비', 28.826499999999996), ('사이즈도', 27.499100000000002), ('색감도', 23.14320000000001), ('여름에', 20.394899999999996), ('생각보다', 19.092999999999996), ('입고', 18.4624), ('사이즈', 16.949499999999997)]


In [10]:
from collections import defaultdict
from konlpy.tag import Okt

# Okt 형태소 분석기 초기화
okt = Okt()

stop_words = ['좋다', '입다', '같다', '이다', '있다', '입', '생각']


# 키워드를 원형으로 변환하는 함수
def lemmatize_keyword(keyword):
    morphs = okt.pos(keyword, stem=True)  # 형태소 분석 + 어간 추출
    # 품사가 명사(Noun)이거나 동사(Verb), 형용사(Adjective)인 경우만 원형으로 변환
    lemmatized = [word for word, pos in morphs if pos in ['Noun', 'Verb', 'Adjective']]
    return ' '.join(lemmatized)  # 원형 변환된 단어들을 문자열로 합침

# 키워드와 유사도를 저장할 딕셔너리 초기화
keyword_scores = defaultdict(float)

# 키워드를 모두 모아서 유사도를 합산 (원형으로 변환하여 처리)
for keywords in df['Keywords']:
    for keyword, score in keywords:
        lemmatized_keyword = lemmatize_keyword(keyword)  # 키워드 원형 변환
        if lemmatized_keyword and lemmatized_keyword not in stop_words:  # 원형이 비어있지 않고 불용어가 아니라면
            keyword_scores[lemmatized_keyword] += score  # 원형 키워드에 유사도 합산

# 유사도가 높은 상위 10개 키워드 추출
top_10_keywords_with_scores = sorted(keyword_scores.items(), key=lambda x: x[1], reverse=True)[:10]

# 결과 출력
print(top_10_keywords_with_scores)


[('사이즈', 68.7178), ('색감', 54.8367), ('만족하다', 37.39309999999999), ('가성 비', 36.63450000000001), ('여름', 34.87310000000001), ('재질', 33.6559), ('배송', 32.547399999999996), ('가격', 32.340700000000005), ('시원하다', 30.535999999999994), ('니트', 29.453899999999997)]
